In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

### Data Import & Reading

In [ ]:
global_temp=pd.read_csv('../input/global-climate-change/GlobalTemperatures.csv')

In [ ]:
global_temp.head()

### Fetching year from date column

In [ ]:
global_temp['dt'][0].split('-')[0]

In [ ]:
def fetch_year(date):
    return date.split('-')[0]

In [ ]:
global_temp['Year']= global_temp['dt'].apply(fetch_year)

In [ ]:
global_temp.head()

In [ ]:
data = global_temp.groupby('Year').agg({'LandAverageTemperature':'mean','LandAverageTemperatureUncertainty':'mean'}).reset_index()

In [ ]:
data.head()

### Creating derieved columns

In [ ]:
## creating top and bottom temp uncertainity

data['Uncertainity_top'] = data['LandAverageTemperature']+ data['LandAverageTemperatureUncertainty']
data['Uncertainity_bottom'] = data['LandAverageTemperature']- data['LandAverageTemperatureUncertainty']

In [ ]:
data.head()

In [ ]:
fig = px.line(data, x='Year', y=['LandAverageTemperature','Uncertainity_top','Uncertainity_bottom'], title='World avg.Land Temp')
fig.show()

#### We can notice that the avg. temp has been increasing in the past 30 years, indicating a rapid increase in global warming.

### Analyzing avg.temperature across different season

In [ ]:
## changing date column to datetime

global_temp['dt'] = pd.to_datetime(global_temp['dt'])

In [ ]:
global_temp.dtypes

In [ ]:
## creating month col from date col
global_temp['month']=global_temp['dt'].dt.month

In [ ]:
## defining function to map seasons on month col

def get_season(month):
    if month >= 3 and month <=4:
        return 'spring'
    elif month >=5 and month <=6:
        return 'summer'
    elif month >=7 and month <=8:
        return 'rainy'
    elif month >=9 and month <=11:
        return 'autumn'
    else:
        return 'winter'

In [ ]:
global_temp['season']=global_temp['month'].apply(get_season)

In [ ]:
global_temp.head()

In [ ]:
## create a list of years
years = global_temp['Year'].unique()

In [ ]:
## create empty season list
spring_temp = []
summer_temp = []
rainy_temp = []
autumn_temp = []
winter_temp = []

In [ ]:
for year in years:
    current_df= global_temp[global_temp['Year']==year]
    spring_temp.append(current_df[current_df['season']=='spring']['LandAverageTemperature'].mean())
    summer_temp.append(current_df[current_df['season']=='summer']['LandAverageTemperature'].mean())
    rainy_temp.append(current_df[current_df['season']=='rainy']['LandAverageTemperature'].mean())
    autumn_temp.append(current_df[current_df['season']=='autumn']['LandAverageTemperature'].mean())
    winter_temp.append(current_df[current_df['season']=='winter']['LandAverageTemperature'].mean())

In [ ]:
## creating a season df to store values of all seasons
season =pd.DataFrame()

In [ ]:
season['year']=years
season['spring_temp'] = spring_temp
season['summer_temp'] = summer_temp
season['rainy_temp'] = rainy_temp
season['autumn_temp'] = autumn_temp
season['winter_temp'] = winter_temp

In [ ]:
season.head()

In [ ]:
fig=px.line(season, x='year', y=['spring_temp', 'summer_temp', 'rainy_temp', 'autumn_temp','winter_temp'],
         title='Avg.temp across season')
fig.show()

#### We can notice that from the past 3-4 decades avg. seasonal temepratures have been increasing worldwide.

### Data Preprocessing
#### For our timeseries model we'll use cities dataset.

In [ ]:
cities = pd.read_csv("../input/global-climate-change/GlobalLandTemperaturesByCity.csv")

In [ ]:
cities.head()

In [ ]:
cities.shape

In [ ]:
## filtering USA data
usa = cities[cities['Country']== 'United States']

In [ ]:
usa.shape

In [ ]:
## extracting specific cities from usa data

data = ['New York','Los Angeles','San Francisco']
data2 = usa[usa['City'].isin(data)]

In [ ]:
data2.head()

In [ ]:
data2=data2[['dt','AverageTemperature']]

In [ ]:
data2.head()

In [ ]:
## renaming columns
data2.columns=['Date','Temperature']

In [ ]:
data2.head()

In [ ]:
data2.dtypes

In [ ]:
## changing date column to datetime
data2['Date']=pd.to_datetime(data2['Date'])

In [ ]:
data2.shape

In [ ]:
data2.isna().sum()

In [ ]:
## since there are very few null values, therefore we'll drops the rows
data2.dropna(inplace=True)

In [ ]:
data2.shape

In [ ]:
## setting the date column as index for timeseries model.
data2.set_index('Date',inplace=True)

In [ ]:
data2.head()

### Data Stability Check

In [ ]:
## lets visulize the data with line plot
sns.lineplot(x=data2.index, y=data2['Temperature'])

#### From the above plot, we are not able to draw concrete inference about data stability, therefore we'll use statistical test to check it.

### Augmented Dickey-Fuller test

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
test_result=adfuller(data2['Temperature'])

In [ ]:
test_result

#### Based on the above test where p-vale is >0.05, we fail to reject the null hypothesis, data has unit root & is Non-stationary

### Making the data stationary

In [ ]:
## making a copy
df = data2.copy()

In [ ]:
df.head()

#### year has a 12 month cycle therefore we are going to shift the temp by 12 places and then calculate the temp difference.

In [ ]:
df['first_temp_diff']= df['Temperature']-df['Temperature'].shift(12)

In [ ]:
df.head(20)

In [ ]:
df.shape

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

In [ ]:
test_result1=adfuller(df['first_temp_diff'])

In [ ]:
test_result1

#### The p-value is almost zero and data seems to be stationary, we can verify it by plotting.

In [ ]:
df[['first_temp_diff']].plot(figsize=(25,12))

### Seasonality Check

In [ ]:
## creating month column
data2['month']=data2.index.month

## creating year column
data2['year']=data2.index.year

In [ ]:
data2.head()

In [ ]:
pivot=data2.pivot_table(values='Temperature', index='month', columns='year')

In [ ]:
pivot

In [ ]:
monthly_seasonality=pivot.mean(axis=1)

In [ ]:
monthly_seasonality.plot(figsize=(20,6))

#### The data shows monthly seasonality as the avg. temperature across all years show a rise in month of June to August.

### Moving Average Method

In [ ]:
## for time series model we need stationary data
df.head()

In [ ]:
## subset the column first_temp_diff
df= df[['first_temp_diff']]

In [ ]:
df.head()

In [ ]:
## caluclating rolling mean
df['first_temp_diff'].rolling(window=5).mean()

In [ ]:
value = pd.DataFrame(df['first_temp_diff'])

In [ ]:
## merging two results
temp_df= pd.concat([value, df['first_temp_diff'].rolling(window=5).mean()],axis=1)

In [ ]:
## renaming columns
temp_df.columns=['actual_temp','forcast_temp']

In [ ]:
temp_df.head()

### Model Evaluation

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
## calulating metrics from 5th row value excluding NaN
np.sqrt(mean_squared_error(temp_df['forcast_temp'][4:],temp_df['actual_temp'][4:]))

#### The above error rate indicates that on any given day there will be a (+/-) 2.39 difference in temp.

### ARIMA Model

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
df.head()

In [ ]:
plot_acf(df['first_temp_diff'].dropna())

In [ ]:
plot_pacf(df['first_temp_diff'].dropna())

In [ ]:
df.isna().sum()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
df.shape

In [ ]:
## train test split

train_data = df[0:6000]
test_data = df[6000:]

In [ ]:
arima = ARIMA(train_data, order=(2,1,3))

In [ ]:
model = arima.fit()

In [ ]:
## forcasting for 10 days on train data

model.forecast(steps=10)[0]

### Forecasting on Test data

In [ ]:
predictions=model.forecast(steps=len(test_data))[0]

In [ ]:
predictions

In [ ]:
from pandas import DataFrame

In [ ]:
predict = pd.DataFrame(predictions, columns=["pred"])

### Model Evaluation

In [ ]:
np.sqrt(mean_squared_error(test_data, predictions))

### Conclusion:
#### Compared to the moving avg method error rate of 2.39, ARIMA error rate is less and therefore a better model for temperature forecast.